# Transformer 번역 모델 데모

이 노트북은 PyTorch로 구현한 Transformer 모델의 영어-한국어 번역 데모입니다.

## 1. 라이브러리 및 모듈 임포트

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os

# 프로젝트 모듈
from src.model import Transformer
from src.data_utils import create_tokenizer, create_data_loader, prepare_sample_data, save_tokenizer

# 설정
plt.style.use('seaborn-v0_8')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. 샘플 데이터 준비 및 탐색

In [ ]:
# 샘플 데이터 로드
src_texts, tgt_texts = prepare_sample_data()

print("샘플 번역 데이터:")
print("=" * 60)
for i, (src, tgt) in enumerate(zip(src_texts, tgt_texts)):
    print(f"{i+1:2d}. EN: {src}")
    print(f"    KO: {tgt}")
    print()

print(f"총 데이터 개수: {len(src_texts)}")

## 3. 토크나이저 생성 및 분석

In [ ]:
# 토크나이저 생성
print("토크나이저 생성 중...")
src_tokenizer = create_tokenizer(src_texts * 10, vocab_size=2000)  # 데이터 확장
tgt_tokenizer = create_tokenizer(tgt_texts * 10, vocab_size=2000)

print(f"영어 어휘 크기: {src_tokenizer.get_vocab_size()}")
print(f"한국어 어휘 크기: {tgt_tokenizer.get_vocab_size()}")

# 토크나이징 예시
example_text = "Hello, how are you?"
encoded = src_tokenizer.encode(example_text)

print(f"\n예시 텍스트: {example_text}")
print(f"토큰 ID: {encoded.ids}")
print(f"토큰: {encoded.tokens}")
print(f"디코딩: {src_tokenizer.decode(encoded.ids)}")

## 4. 모델 아키텍처 분석

In [ ]:
# 모델 생성
model = Transformer(
    src_vocab_size=src_tokenizer.get_vocab_size(),
    tgt_vocab_size=tgt_tokenizer.get_vocab_size(),
    d_model=256,
    n_heads=8,
    n_layers=4,
    d_ff=1024,
    max_seq_length=128
).to(device)

# 모델 정보 출력
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("모델 정보:")
print(f"총 파라미터 수: {total_params:,}")
print(f"학습 가능한 파라미터 수: {trainable_params:,}")
print(f"모델 크기 (MB): {total_params * 4 / (1024**2):.2f}")

# 모델 구조 출력
print("\n모델 구조:")
print(model)

## 5. 데이터 로더 생성 및 배치 분석

In [ ]:
# 데이터 로더 생성
train_loader = create_data_loader(
    src_texts * 5, tgt_texts * 5,  # 데이터 확장
    src_tokenizer, tgt_tokenizer,
    batch_size=4, max_length=64
)

# 첫 번째 배치 분석
for batch in train_loader:
    print("배치 정보:")
    print(f"Source shape: {batch['src'].shape}")
    print(f"Target input shape: {batch['tgt_input'].shape}")
    print(f"Target output shape: {batch['tgt_output'].shape}")
    
    # 첫 번째 샘플 디코딩
    src_decoded = src_tokenizer.decode(batch['src'][0].numpy())
    tgt_decoded = tgt_tokenizer.decode(batch['tgt_input'][0].numpy())
    
    print(f"\n첫 번째 샘플:")
    print(f"Source: {src_decoded}")
    print(f"Target: {tgt_decoded}")
    break

## 6. 모델 순전파 테스트

In [ ]:
# 모델 순전파 테스트
model.eval()
with torch.no_grad():
    for batch in train_loader:
        src = batch['src'].to(device)
        tgt_input = batch['tgt_input'].to(device)
        
        output = model(src, tgt_input)
        
        print("순전파 테스트 결과:")
        print(f"입력 크기: {src.shape}")
        print(f"출력 크기: {output.shape}")
        print(f"예상 출력 크기: (batch_size={src.shape[0]}, seq_len={tgt_input.shape[1]}, vocab_size={tgt_tokenizer.get_vocab_size()})")
        
        # 확률 분포 확인
        probs = torch.softmax(output[0, -1, :], dim=-1)
        top5_probs, top5_indices = torch.topk(probs, 5)
        
        print("\n마지막 위치 상위 5개 토큰 확률:")
        for i, (prob, idx) in enumerate(zip(top5_probs, top5_indices)):
            token = tgt_tokenizer.decode([idx.item()])
            print(f"{i+1}. {token}: {prob.item():.4f}")
        
        break

## 7. 학습 프로세스 시뮬레이션 (짧은 학습)

In [ ]:
# 짧은 학습 세션
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=0)  # 패딩 토큰 무시

losses = []
num_epochs = 5

print("짧은 학습 시작...")
for epoch in range(num_epochs):
    epoch_loss = 0
    num_batches = 0
    
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        src = batch['src'].to(device)
        tgt_input = batch['tgt_input'].to(device)
        tgt_output = batch['tgt_output'].to(device)
        
        optimizer.zero_grad()
        
        output = model(src, tgt_input)
        loss = criterion(output.view(-1, output.size(-1)), tgt_output.view(-1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        num_batches += 1
    
    avg_loss = epoch_loss / num_batches
    losses.append(avg_loss)
    print(f"Epoch {epoch+1}, Average Loss: {avg_loss:.4f}")

print("짧은 학습 완료!")

## 8. 학습 곡선 시각화

In [ ]:
# 학습 곡선 그리기
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(losses)+1), losses, 'b-', marker='o', linewidth=2, markersize=6)
plt.title('Training Loss Curve', fontsize=16, fontweight='bold')
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(range(1, len(losses)+1))

# 손실 값 표시
for i, loss in enumerate(losses):
    plt.annotate(f'{loss:.3f}', (i+1, loss), textcoords="offset points", 
                xytext=(0,10), ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print(f"최종 손실: {losses[-1]:.4f}")
print(f"손실 감소: {losses[0] - losses[-1]:.4f} ({((losses[0] - losses[-1])/losses[0]*100):.1f}%)")

## 9. 간단한 번역 테스트

In [ ]:
def simple_translate(model, text, src_tokenizer, tgt_tokenizer, max_length=64):
    """간단한 greedy 디코딩 번역"""
    model.eval()
    with torch.no_grad():
        # 입력 텍스트 인코딩
        src_encoding = src_tokenizer.encode(text)
        src_ids = src_encoding.ids[:max_length-1] + [src_tokenizer.token_to_id("[EOS]")]
        src_ids += [src_tokenizer.token_to_id("[PAD]")] * (max_length - len(src_ids))
        src = torch.tensor(src_ids[:max_length], dtype=torch.long).unsqueeze(0).to(device)
        
        # 디코더 시작 토큰
        tgt = torch.tensor([[tgt_tokenizer.token_to_id("[BOS]")]], dtype=torch.long).to(device)
        
        # 순차적 디코딩
        for _ in range(max_length - 1):
            output = model(src, tgt)
            next_token = torch.argmax(output[0, -1, :], dim=-1)
            tgt = torch.cat([tgt, next_token.unsqueeze(0).unsqueeze(0)], dim=1)
            
            if next_token.item() == tgt_tokenizer.token_to_id("[EOS]"):
                break
        
        # 결과 디코딩
        result_ids = tgt[0].cpu().numpy()[1:]  # BOS 토큰 제거
        result_ids = [id for id in result_ids if id not in [0, tgt_tokenizer.token_to_id("[EOS]")]]  # 패딩과 EOS 제거
        
        if result_ids:
            return tgt_tokenizer.decode(result_ids)
        return "[번역 실패]"

# 번역 테스트
test_sentences = [
    "Hello, how are you?",
    "I love machine learning.",
    "The weather is nice today.",
    "Thank you very much."
]

print("번역 테스트 결과:")
print("=" * 60)

for sentence in test_sentences:
    translation = simple_translate(model, sentence, src_tokenizer, tgt_tokenizer)
    print(f"EN: {sentence}")
    print(f"KO: {translation}")
    print("-" * 40)

## 10. 어텐션 가중치 시각화

In [ ]:
def get_attention_weights(model, src, tgt):
    """어텐션 가중치 추출 (간단한 버전)"""
    model.eval()
    with torch.no_grad():
        # 첫 번째 인코더 레이어의 어텐션 가중치만 추출
        src_embedded = model.src_embedding(src) * (model.d_model ** 0.5)
        src_embedded = model.positional_encoding(src_embedded.transpose(0, 1)).transpose(0, 1)
        
        # 첫 번째 인코더 레이어의 self-attention
        encoder_layer = model.encoder_layers[0]
        
        # Multi-head attention에서 query, key, value 계산
        batch_size = src.size(0)
        Q = encoder_layer.self_attention.W_q(src_embedded).view(batch_size, -1, 8, 32).transpose(1, 2)
        K = encoder_layer.self_attention.W_k(src_embedded).view(batch_size, -1, 8, 32).transpose(1, 2)
        
        # 어텐션 스코어 계산
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (32 ** 0.5)
        attention_weights = torch.softmax(scores, dim=-1)
        
        return attention_weights

# 샘플 문장으로 어텐션 시각화
sample_text = "Hello, how are you?"
src_encoding = src_tokenizer.encode(sample_text)
src_ids = src_encoding.ids[:10] + [0] * (20 - len(src_encoding.ids[:10]))
src = torch.tensor(src_ids, dtype=torch.long).unsqueeze(0).to(device)
tgt = torch.tensor([[tgt_tokenizer.token_to_id("[BOS]")]], dtype=torch.long).to(device)

try:
    attention_weights = get_attention_weights(model, src, tgt)
    
    # 첫 번째 헤드의 어텐션 가중치 시각화
    attn_matrix = attention_weights[0, 0, :len(src_encoding.ids), :len(src_encoding.ids)].cpu().numpy()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(attn_matrix, 
                xticklabels=src_encoding.tokens[:len(src_encoding.ids)],
                yticklabels=src_encoding.tokens[:len(src_encoding.ids)],
                annot=True, fmt='.3f', cmap='Blues')
    plt.title('Self-Attention Weights (First Head)', fontsize=14, fontweight='bold')
    plt.xlabel('Key Tokens', fontsize=12)
    plt.ylabel('Query Tokens', fontsize=12)
    plt.tight_layout()
    plt.show()
    
except Exception as e:
    print(f"어텐션 시각화 중 오류 발생: {e}")
    print("모델이 충분히 학습되지 않았거나 구조상 문제가 있을 수 있습니다.")

## 11. 모델 저장

In [ ]:
# 디렉토리 생성
os.makedirs('demo_checkpoints', exist_ok=True)
os.makedirs('demo_tokenizers', exist_ok=True)

# 모델 저장
torch.save({
    'model_state_dict': model.state_dict(),
    'src_vocab_size': src_tokenizer.get_vocab_size(),
    'tgt_vocab_size': tgt_tokenizer.get_vocab_size(),
    'losses': losses
}, 'demo_checkpoints/demo_model.pth')

# 토크나이저 저장
save_tokenizer(src_tokenizer, 'demo_tokenizers/src_tokenizer.json')
save_tokenizer(tgt_tokenizer, 'demo_tokenizers/tgt_tokenizer.json')

print("모델과 토크나이저가 저장되었습니다!")
print("- 모델: demo_checkpoints/demo_model.pth")
print("- 소스 토크나이저: demo_tokenizers/src_tokenizer.json")
print("- 타겟 토크나이저: demo_tokenizers/tgt_tokenizer.json")

## 결론

이 데모에서는 다음을 수행했습니다:

1. **데이터 준비**: 영어-한국어 번역 샘플 데이터 생성
2. **토크나이저**: BPE 토크나이저로 텍스트를 토큰화
3. **모델 구축**: Transformer 아키텍처 구현 및 분석
4. **학습**: 짧은 학습 세션으로 모델 훈련
5. **평가**: 간단한 번역 테스트 수행
6. **시각화**: 학습 곡선과 어텐션 가중치 시각화

### 개선 방향:
- 더 큰 데이터셋 사용
- 더 긴 학습 시간
- 하이퍼파라미터 튜닝
- Beam search 디코딩
- BLEU 점수 등 정량적 평가 지표 추가

전체 학습을 위해서는 `train.py` 스크립트를 실행하세요!